# TP2 Optimisation modèle : Réclamation aux Etats Unis

## Preprocessing

### Import de librairies et lecture du fichier

In [1]:
# Librairie pour la manipulation des dataframes
import pandas as pd
# librairie pour la manipulation des tableaux (array)
import numpy as np

# libraires (os & glob) pour la manipulation de l'os (systeme d'exploitatin). Elle permet de se balader dans l'ordinateur
import os
import glob

# gc est le garbage collector (pr gérer la mémoire vive de son pc), c'est très important quand on multiplie le nombre de table
import gc

# sklearn : scikit-learn est la librairie utilisé par la majorité des utilisateurs pour toutes les fonctionnalités de base de datascience.
#elle contient beaucoup de fonctionnalité très pratique
# preprocessing : La normalisation ou la standardisation
from sklearn import preprocessing
#Metrics permet de programmer automatiquement toutes les métriques usuelles
from sklearn import metrics

# train_test_split => Fonctionnalité très utile pour créer ses datasets de test et de train
from sklearn.model_selection import train_test_split

# xgboost => librairie optimisé utilisant le model de descente du gradient optimisé: http://xgboost.readthedocs.io/en/latest/model.html 
import xgboost as xgb

In [2]:
#libraire de NLP classic (spacy est plus à la mode aujourd'hui)
import nltk
#utile pour "tokenizer" les zones de texte : rappel tokenizer('Bonjour, comment allez vous?') = [Bonjour, comment, allez, vous]
from nltk.tokenize import RegexpTokenizer, sent_tokenize, word_tokenize
#utile pour le pos_tagging ou lemmatizer (corpus définit les dictionnaires de vocabulaire créer par les auteurs de nltk)
from nltk.corpus import wordnet as wn
# import le dictionnaire de mot usuel qui n'ont pas vocation à pouvoir permettre d'apporter de l'information pour la prise de décision
from nltk.corpus import stopwords
# lemmatizer(remove_stop_word(nous avons beaucoup de travaux à finir) = beaucoup travail finir
from nltk.stem import WordNetLemmatizer

In [3]:
#création d'une instance de tokenization
tokenizer = RegexpTokenizer(r'\w+')
# création d'une instance de lemmatization
lemmatizer = WordNetLemmatizer()

In [11]:
#creation d'une variable de contexte pour le folder
s_working_directory = '/Users/moi/Documents/CoursDauphine/module3/projects/finance'

In [5]:
# téléchargement de tous les mots packages de base pour nltk
# nltk.download('popular')

In [5]:
#Run du fichier toolkit : 
%run "/Users/moi/Documents/CoursDauphine/module3/projects/finance/TP3 FI/toolkit.py"

### Lecture de la table des variables créer pendant le TP2 et réapplication de la fonction de la lemmatization

In [13]:
df_complaint = pd.read_csv(s_working_directory + '/us-consumer-finance-complaint-database/TP_2_var_consumer_complaints.csv', sep=';', decimal='.',encoding='utf-8')

In [14]:
df_complaint.head()

,index,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,...,var_6_nb_reclamation_state_sub_issue,var_7_nb_reclamation_state_sub_product,mean|var_1_bis_taille_reclamation_sub_product,std|var_1_bis_taille_reclamation_sub_product,var_8_distrib_normal_var_1_x_sub_produit,mean|var_1_bis_taille_reclamation_sub_issue,std|var_1_bis_taille_reclamation_sub_issue,var_9_distrib_normal_var_1_x_sub_issue,var_10_nb_XXXX,var_11_nb_$
0,190126,2015-03-19,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt was paid,XXXX has claimed I owe them {$27.00} for XXXX ...,NaN,"Diversified Consultants, Inc.",NY,...,105.0,237.0,60.811156,44.655486,-0.300200,59.765306,41.898063,-0.307476,2,2
1,196219,2015-03-26,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt was paid,I have informed credit reporting agencies abou...,NaN,"Hunter Warfield, Inc.",HI,...,3.0,12.0,60.811156,44.655486,0.394004,59.765306,41.898063,0.432415,9,2
2,197286,2015-03-26,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt was paid,"I resided at XXXX from XXXX ( see attached ), ...",Company believes complaint is the result of an...,First Advantage LNS Inc.,TX,...,200.0,592.0,60.811156,44.655486,-0.210625,59.765306,41.898063,-0.212006,7,2
3,197411,2015-03-20,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt was paid,XXXX apartments is trying to collect an amount...,Company believes it acted appropriately as aut...,"I.Q. Data International, Inc.",CA,...,300.0,681.0,60.811156,44.655486,0.304429,59.765306,41.898063,0.336945,5,2
4,199217,2015-03-27,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt was paid,XXXX has sent a previous account into collecti...,Company believes it acted appropriately as aut...,"Diversified Consultants, Inc.",AL,...,28.0,66.0,60.811156,44.655486,-0.423365,59.765306,41.898063,-0.438747,1,2


In [15]:
# replace _ par espace (contenu dans 89 lignes)
df_complaint['ins_consumer_complaint_narrative_stop_words_lemmatized'] = df_complaint['ins_consumer_complaint_narrative_stop_words_lemmatized'].str.replace('_', ' ')
# mask excluant les textes vide après exclusion des stops words
mask = df_complaint['ins_consumer_complaint_narrative_stop_words_lemmatized'].notnull()
# application du mask pour filter puis reinitialisation de l'index (voir dans le fichier .py)
df_complaint = _reinitialize_df(df_complaint[mask])

TypeError: drop() got an unexpected keyword argument 'columns'

In [37]:
df_complaint['ins_consumer_complaint_narrative_stop_words_lemmatized'] = df_complaint['ins_consumer_complaint_narrative_stop_words_lemmatized'].apply(_lemmatize)
df_complaint = _reinitialize_df(df_complaint[~df_complaint['ins_consumer_complaint_narrative_stop_words_lemmatized'].str.contains('_')])

In [38]:
# Création d'une liste ou chaque élement est le commentaire d'une réclamation
list_words_content = df_complaint.ins_consumer_complaint_narrative_stop_words_lemmatized.tolist()
# Création d'une liste de tous les mots des commentaires juxtaposé les un avec les autres 
list_words_all_content = ' '.join(tokenizer.tokenize(' '.join(list_words_content).lower().replace(',',' ').replace('-',' ')))
# Création du contenu tokenizer de la liste
all_content_tokenized = tokenizer.tokenize(list_words_all_content)
# Création de la table contenant tous les mots avec leur fréquence
df_sorted_content = _vocabulary_frequence(all_content_tokenized)
# apparition de la table
df_sorted_content

,Ins_Word,Ins_Nb_Appearance
0,credit,89200
1,account,84975
2,call,72009
3,payment,69291
4,pay,57704
5,loan,55537
6,report,53244
7,bank,45944
8,tell,45268
9,receive,43279


In [40]:
set_words = set(all_content_tokenized)
print(len(set_words))
print(len(all_content_tokenized))

40419
5273387


### Création de la table intermédiaire pour l'analyse des mots d'importance pour la prédiction

In [41]:
#Création du y_output avec 1 si Oui 0 sinon
df_complaint['y_output'] = np.where(df_complaint['consumer_disputed?'] == 'Yes', 1, 0)

In [58]:
#Petite astuce pour créer les colonnes automatiquement issue d'un split entre les mots
l_col_name = ['word_' + str(value) for value in pd.DataFrame(df_complaint['ins_consumer_complaint_narrative_stop_words_lemmatized'].str.split(' ').tolist()).columns]
# creation d'une table temporaire qui va contenir une colonne par mots de la réclamation et le y_output ainsi que le y output
df_tmp = df_complaint[['y_output', 'complaint_id']].merge(pd.DataFrame(df_complaint['ins_consumer_complaint_narrative_stop_words_lemmatized'].str.split(' ').tolist(), columns=l_col_name), left_index=True, right_index=True)

In [60]:
df_melt = pd.melt(df_tmp, id_vars=['complaint_id', 'y_output'], var_name='variable_name', value_vars=l_col_name, value_name='word')
df_melt = df_melt[df_melt.word.notnull()]

In [65]:
del df_tmp
gc.collect()

436

In [66]:
# création d'un pivot table avec deux fonctions d'aggrégation : somme et nombre de valeurs
df_pivot = pd.pivot_table(df_melt, values='y_output', index='word', aggfunc=[np.sum, len])
# peite astuce pour combien le nom des colonnes afin de ne pas avoir de problème lors du reset_index
df_pivot.columns = ['%s%s' % (a, '|%s' % b if b else '') for a, b in df_pivot.columns]
df_pivot

,sum|y_output,len|y_output
word,,
a,3,9
aa,19,40
aaa,4,21
aaaked,0,1
aaarm,0,2
aac,0,1
aacceptance,0,1
aaccount,0,2
aaccounts,0,1


In [67]:
# reset_index afin d'éviter les complexités de manipulation de niveau des axes (pour les colonnes) et de l'index ligne qui sera le mot
df_pivot = df_pivot.reset_index()

In [69]:
# Fonction pour renommer les colonnes
df_pivot = df_pivot.rename(columns={'sum|y_output' : 'nb_yes', 
                                    'len|y_output' : 'nb_occurence'})

In [70]:
df_pivot['ins_ratio_yes'] = df_pivot['nb_yes'] / df_pivot['nb_occurence']

L'idée de construire ce lexique est de pouvoir compter le nombre de mots de ce lexique (sous forme de combinaison) qui sera présent dans la réclamation

In [71]:
# Définition du lexique de mots qui vont être considéré comme n'entrainant pas de réclamations
l_no_dispute = df_pivot[(df_pivot['ins_ratio_yes'] < 0.05)]['word'].tolist()
set_no_dispute = set(l_no_dispute)

L'idée de construire ce lexique est de pouvoir compter le nombre de mots de ce lexique (sous forme de combinaison) qui sera présent dans la réclamation

In [72]:
l_disputed = df_pivot[(df_pivot['ins_ratio_yes'] > 0.35)]['word'].tolist() 
set_disputed = set(l_disputed)

In [73]:
# Pas utilisé au final car n'a pas apporté de valeurs
l_all_word_50 = df_pivot[(df_pivot['nb_occurence'] > 50)]['word'].tolist()
set_all_word_50 = set(l_all_word_50)

### Création des variables

In [85]:
# Création des nouvelles variables pour l'utilisation du modèle: 
df_complaint['var12_nb_word_dispute'] = df_complaint['ins_consumer_complaint_narrative_stop_words_lemmatized'].apply(_nb_word_dispute)
df_complaint['var13_nb_word_different'] = df_complaint['ins_consumer_complaint_narrative_stop_words_lemmatized'].apply(lambda x : len(set(tokenizer.tokenize(x))))
df_complaint['va14_ratio_word_dispute_vs_all'] =  df_complaint['var12_nb_word_dispute'] / df_complaint['var13_nb_word_different'] 
df_complaint['var15_nb_word_positif'] = df_complaint['ins_consumer_complaint_narrative_stop_words_lemmatized'].apply(lambda x : len(set(tokenizer.tokenize(x)).intersection(set_no_dispute)))
df_complaint['va16_ratio_word_positif_vs_all'] =  df_complaint['var15_nb_word_positif'] / df_complaint['var13_nb_word_different']
df_complaint['va17_ratio_word_negatif_vs_word_positif'] = df_complaint['var12_nb_word_dispute'] / np.where(df_complaint['var15_nb_word_positif'] == 0, 1, df_complaint['var15_nb_word_positif'])

In [86]:
df_complaint[['var12_nb_word_dispute', 'var13_nb_word_different', 'va14_ratio_word_dispute_vs_all', 'var15_nb_word_positif', 'va16_ratio_word_positif_vs_all', 'va17_ratio_word_negatif_vs_word_positif', 'y_output']].to_csv(s_working_directory+ '/data.csv', sep=';')

## Modèlisation :

### création des deux datasets (un avec les nouvelles variables et un avec ceux de base)

In [77]:
df_complaint.columns

Index(['date_received', 'product', 'sub_product', 'issue', 'sub_issue',
       'consumer_complaint_narrative', 'company_public_response', 'company',
       'state', 'zipcode', 'tags', 'consumer_consent_provided',
       'submitted_via', 'date_sent_to_company', 'company_response_to_consumer',
       'timely_response', 'consumer_disputed?', 'complaint_id', 'tec_count',
       'ins_consumer_complaint_narrative_stop_words',
       'ins_consumer_complaint_narrative_stop_words_lemmatized',
       'var_1_taille_reclamation', 'var_1_bis_taille_reclamation',
       'var_2_nb_reclamation_company',
       'var_3_nb_reclamation_company_sub_product',
       'var_4_nb_reclamation_company_sub_issue',
       'var_5_nb_reclamation_company_state',
       'var_6_nb_reclamation_state_sub_issue',
       'var_7_nb_reclamation_state_sub_product',
       'mean|var_1_bis_taille_reclamation_sub_product',
       'std|var_1_bis_taille_reclamation_sub_product',
       'var_8_distrib_normal_var_1_x_sub_produit',
  

In [92]:
# df_complaint.to_excel(s_working_directory + '_data/analyse_complaints/TP_2_var_consumer_complaints.xlsx',encoding='utf-8', index=False)

In [87]:
# Création d'une instance de standardisation : 
std_scaler = preprocessing.StandardScaler()

In [88]:
#[[]] permet de faire une sous sélection de variable pour une nouvelle table
df_complaint_dataset_1 = df_complaint[['var_1_bis_taille_reclamation', 
                                       'var_8_distrib_normal_var_1_x_sub_produit', 
                                       'var_9_distrib_normal_var_1_x_sub_issue', 
                                       'var_10_nb_XXXX', 
                                       'var_11_nb_$', 
                                       'var12_nb_word_dispute', 
                                       'var13_nb_word_different', 
                                       'va14_ratio_word_dispute_vs_all', 
                                       'var15_nb_word_positif', 
                                       'va16_ratio_word_positif_vs_all', 
                                       'va17_ratio_word_negatif_vs_word_positif']]
#permet de remplacer les valeurs vides par 0
df_complaint_dataset_1 = df_complaint_dataset_1.fillna(0)

In [92]:
#permet de créer les variables d'observations
df_complaint_dataset_1.describe()

,var_1_bis_taille_reclamation,var_8_distrib_normal_var_1_x_sub_produit,var_9_distrib_normal_var_1_x_sub_issue,var_10_nb_XXXX,var_11_nb_$,var12_nb_word_dispute,var13_nb_word_different,va14_ratio_word_dispute_vs_all,var15_nb_word_positif,va16_ratio_word_positif_vs_all,va17_ratio_word_negatif_vs_word_positif
count,66804.000000,66804.000000,66804.000000,66804.000000,66804.000000,66804.000000,66804.000000,66804.000000,66804.000000,66804.000000,66804.000000
mean,72.096267,0.000020,0.000019,9.498039,1.993488,1.862897,51.608496,0.031489,0.432265,0.008551,1.697211
std,52.199778,0.499826,0.499504,13.636790,0.080432,3.089907,37.656997,0.042366,1.280961,0.024404,2.911040
min,1.000000,-0.812526,-0.956713,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,34.000000,-0.365992,-0.360203,2.000000,2.000000,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000
50%,58.000000,-0.130433,-0.128594,5.000000,2.000000,1.000000,41.000000,0.020408,0.000000,0.000000,1.000000
75%,96.000000,0.233663,0.231672,12.000000,2.000000,2.000000,69.000000,0.046512,0.000000,0.000000,2.000000
max,356.000000,3.617114,3.890187,414.000000,2.000000,55.000000,237.000000,1.000000,44.000000,0.666667,55.000000


In [ ]:
np_dataset_1 = std_scaler.fit_transform(df_complaint_dataset_1)
df_np_dataset_1 = pd.DataFrame(data=np_dataset_1)
np_dataset_1.shape

In [ ]:
pd.DataFrame(data=np_dataset_1).mean()

In [110]:
df_complaint_dataset_2 = df_complaint[['var_1_bis_taille_reclamation',
       'var_2_nb_reclamation_company',
       'var_3_nb_reclamation_company_sub_product',
       'var_4_nb_reclamation_company_sub_issue',
       'var_5_nb_reclamation_company_state',
       'var_6_nb_reclamation_state_sub_issue',
       'var_7_nb_reclamation_state_sub_product', 
       'var_8_distrib_normal_var_1_x_sub_produit',
       'var_9_distrib_normal_var_1_x_sub_issue',
       'var_10_nb_XXXX', 'var_11_nb_$']]
df_complaint_dataset_2 = df_complaint_dataset_2.fillna(0)

In [111]:
np_dataset_2 = std_scaler.fit_transform(df_complaint_dataset_2)
df_np_dataset_2 = pd.DataFrame(data=np_dataset_2)
np_dataset_2.shape

(66804, 11)

In [112]:
pd.DataFrame(data=np_dataset_2).mean()

0     1.323445e-15
1    -2.928025e-14
2    -1.968632e-14
3    -2.431572e-14
4    -1.091820e-14
5     4.280119e-16
6    -1.525083e-14
7    -7.078744e-17
8    -1.120128e-18
9    -3.255940e-15
10    8.731286e-16
dtype: float64

In [96]:
df_complaint['y_to_predict'] = np.where(df_complaint['consumer_disputed?'] == 'Yes', 1, 0)
y_to_predict = df_complaint['y_to_predict']
y_to_predict

0        0
1        0
2        1
3        0
4        0
5        1
6        1
7        1
8        0
9        1
10       0
11       1
12       0
13       0
14       1
15       0
16       0
17       1
18       0
19       0
20       1
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
66774    0
66775    0
66776    0
66777    0
66778    1
66779    0
66780    0
66781    1
66782    0
66783    0
66784    0
66785    1
66786    0
66787    1
66788    0
66789    0
66790    1
66791    0
66792    1
66793    1
66794    1
66795    1
66796    0
66797    0
66798    0
66799    0
66800    1
66801    0
66802    1
66803    0
Name: y_to_predict, Length: 66804, dtype: int32

### Random Forest

In [97]:
from sklearn.ensemble import RandomForestClassifier


C:\HOMEWARE\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Avec le dataset des nouvelles variables

In [117]:
clf = RandomForestClassifier(n_estimators=1000, max_depth=4,
                              random_state=0)
X_train, X_test, y_train, y_test = train_test_split(np_dataset_1, y_to_predict, test_size=0.3, random_state=50)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [118]:
pred_binary_rdmf = clf.predict(X_test)
sum(pred_binary_rdmf)

2085

In [121]:
_print_eval_model(y_test, pred_binary_rdmf)

Confusion matrix : 
[[14710   661]
 [ 3247  1424]]
Accuracy : 0.8050094800918072
Precision : 0.6829736211031175
Recall : 0.30485977306786555


Avec le deuxième dataset

In [113]:
clf2 = RandomForestClassifier(n_estimators=1000, max_depth=4,
                              random_state=0)
X_train2, X_test2, y_train2, y_test2 = train_test_split(np_dataset_2, y_to_predict, test_size=0.3, random_state=50)
clf.fit(X_train2, y_train2)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=4000, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [114]:
pred_binary_rdmf2 = clf2.predict(X_test2)
sum(pred_binary_rdmf2)

0

In [115]:
_print_eval_model(y_test2, pred_binary_rdmf2)

Confusion matrix : 
[[15371     0]
 [ 4671     0]]
Accuracy : 0.766939427202874
Precision : 0.0
Recall : 0.0


C:\HOMEWARE\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### XGBOOST

##### Idée de création des paramètres du modèle

In [103]:
param = {'max_depth':5, 'eta':0.15, 'objective':'binary:logistic', 'eval_metric': 'auc','lambda':1.5, 'silent': 1} #,'gamma':1.5
y_output, pred, bst = calcul_model(df_np_dataset_1, 0.3, y_to_predict, param, 600, 50)

Idée pour visualiser la classification selon la valeur de la probabilité

In [104]:
#méthode d'affichage des valeurs
for value_pred in np.arange(0.1,1,0.1):    
    pred_binary =  np.where(pred >= value_pred, 1, 0)     
    print('p-classification : ' + str(value_pred))
    print('Confusion matrix : \r\n' + str(metrics.confusion_matrix(y_output.tolist(), pred_binary)))
    print('Accuracy : ' + str(metrics.accuracy_score(y_output.tolist(), pred_binary)))
    print('Precision : ' + str(metrics.precision_score(y_output.tolist(), pred_binary)))
    print('Recall : ' + str(metrics.recall_score(y_output.tolist(), pred_binary)))
    print()

p-classification : 0.1
Confusion matrix : 
[[6208 9163]
 [ 336 4335]]
Accuracy : 0.5260453048597944
Precision : 0.3211586901763224
Recall : 0.9280667951188183

p-classification : 0.2
Confusion matrix : 
[[10010  5361]
 [ 1288  3383]]
Accuracy : 0.6682466819678675
Precision : 0.38689387008234216
Recall : 0.7242560479554699

p-classification : 0.30000000000000004
Confusion matrix : 
[[12399  2972]
 [ 2072  2599]]
Accuracy : 0.7483285101287297
Precision : 0.4665230658768623
Recall : 0.5564119032327125

p-classification : 0.4
Confusion matrix : 
[[13644  1727]
 [ 2674  1997]]
Accuracy : 0.7804111366131125
Precision : 0.5362513426423201
Recall : 0.42753157782059514

p-classification : 0.5
Confusion matrix : 
[[14382   989]
 [ 3133  1538]]
Accuracy : 0.7943319030036923
Precision : 0.6086268302334784
Recall : 0.3292656818668379

p-classification : 0.6
Confusion matrix : 
[[14809   562]
 [ 3521  1150]]
Accuracy : 0.7962778165851712
Precision : 0.6717289719626168
Recall : 0.24619995718261614

p

In [106]:
# example si on fait varier le paramètre de la profondeur, l'impact que ça peut avoir.
param = {'max_depth':7, 'eta':0.3, 'objective':'binary:logistic', 'eval_metric': 'auc','lambda':1.5, 'silent': 1} #,'gamma':1.5
y_output, pred, bst = calcul_model(df_np_dataset_1, 0.3, y_to_predict, param, 600, 50)

In [107]:
for value_pred in np.arange(0.1,1,0.1):
    pred_binary =  np.where(pred >= value_pred, 1, 0)     
    print('p-classification : ' + str(value_pred))
    print('Confusion matrix : \r\n' + str(metrics.confusion_matrix(y_output.tolist(), pred_binary)))
    print('Accuracy : ' + str(metrics.accuracy_score(y_output.tolist(), pred_binary)))
    print('Precision : ' + str(metrics.precision_score(y_output.tolist(), pred_binary)))
    print('Recall : ' + str(metrics.recall_score(y_output.tolist(), pred_binary)))
    print()

p-classification : 0.1
Confusion matrix : 
[[7421 7950]
 [ 656 4015]]
Accuracy : 0.5706017363536573
Precision : 0.3355620559966569
Recall : 0.8595589809462642

p-classification : 0.2
Confusion matrix : 
[[10450  4921]
 [ 1492  3179]]
Accuracy : 0.6800219538968166
Precision : 0.3924691358024691
Recall : 0.6805823164204667

p-classification : 0.30000000000000004
Confusion matrix : 
[[12338  3033]
 [ 2159  2512]]
Accuracy : 0.7409440175631175
Precision : 0.45302073940486925
Recall : 0.5377863412545494

p-classification : 0.4
Confusion matrix : 
[[13443  1928]
 [ 2648  2023]]
Accuracy : 0.7716794731064764
Precision : 0.5120222728423184
Recall : 0.4330978377221152

p-classification : 0.5
Confusion matrix : 
[[14124  1247]
 [ 3014  1657]]
Accuracy : 0.7873964674184213
Precision : 0.5705922865013774
Recall : 0.35474202526225646

p-classification : 0.6
Confusion matrix : 
[[14577   794]
 [ 3359  1312]]
Accuracy : 0.7927851511825167
Precision : 0.6229819563152896
Recall : 0.28088203810747164

p

In [122]:
param = {'max_depth':7, 'eta':0.3, 'objective':'binary:logistic', 'eval_metric': 'auc','lambda':1.5, 'silent': 1} #,'gamma':1.5
y_output_2, pred_2, bst = calcul_model(df_np_dataset_2, 0.3, y_to_predict, param, 600, 50)

In [ ]:
for eta in np.arange(0.1:3:0.05):
    param = {'max_depth':7, 'eta':eta, 'objective':'binary:logistic', 'eval_metric': 'auc','lambda':1.5, 'silent': 1} #,'gamma':1.5
    y_output_2, pred_2, bst = calcul_model(df_np_dataset_2, eta, y_to_predict, param, 600, 50)
    

In [125]:
for value_pred_2 in np.arange(0.1,1,0.1):
    pred_binary2 =  np.where(pred_2 >= value_pred_2, 1, 0)  
    print('p : ' + str(value_pred_2))
    _print_eval_model(y_output, pred_binary2)

p : 0.1
Confusion matrix : 
[[6294 9077]
 [1430 3241]]
Accuracy : 0.4757509230615707
Precision : 0.26311089462575094
Recall : 0.6938557054163991
p : 0.2
Confusion matrix : 
[[9996 5375]
 [2566 2105]]
Accuracy : 0.6037820576788744
Precision : 0.28141711229946526
Recall : 0.4506529651038322
p : 0.30000000000000004
Confusion matrix : 
[[12160  3211]
 [ 3274  1397]]
Accuracy : 0.6764294980540864
Precision : 0.3031684027777778
Recall : 0.2990794262470563
p : 0.4
Confusion matrix : 
[[13454  1917]
 [ 3740   931]]
Accuracy : 0.7177427402454845
Precision : 0.32689606741573035
Recall : 0.19931492185827446
p : 0.5
Confusion matrix : 
[[14242  1129]
 [ 4061   610]]
Accuracy : 0.7410438080031932
Precision : 0.35077630822311673
Recall : 0.13059302076643117
p : 0.6
Confusion matrix : 
[[14760   611]
 [ 4270   401]]
Accuracy : 0.7564614309949107
Precision : 0.39624505928853754
Recall : 0.0858488546349818
p : 0.7000000000000001
Confusion matrix : 
[[15081   290]
 [ 4434   237]]
Accuracy : 0.7642949805

Question : Quelle variable pourrions nous créer? 

Question : Comment pourrions-nous optimiser le modèle?

Méthode naive de recherche de valeur qui optimise la précision pour un paramètre

In [ ]:
l_mode_optim = []
l_xgb_accuracy = []
l_xgb_precision = []
l_xgb_recall = []
l_best_param = []

In [ ]:
xgb_precision = 0
xgb_accuracy = 0
xgb_recall = 0
best_param = 0
for value_param in range(1,10,1):
    print(value_param)
    param = {'max_depth':value_param, 'eta':0.3, 'silent': 1, 'objective':'binary:logistic', 'eval_metric': 'auc','lambda':1.5} #,'gamma':1.5
    y_output, pred, bst = calcul_model(df_np_dataset, 0.3, y_to_predict, param, 300, 50)
    pred_binary =  np.where(pred > 0.5, 1, 0) 
    if metrics.precision_score(y_output.tolist(), pred_binary) > xgb_precision:
        xgb_accuracy = metrics.accuracy_score(y_output.tolist(), pred_binary)
        precision_confusion_metrics = metrics.confusion_matrix(y_output.tolist(), pred_binary)
        best_param = value_param
        xgb_precision = metrics.precision_score(y_output.tolist(), pred_binary)
        xgb_recall = metrics.recall_score(y_output.tolist(), pred_binary)

In [ ]:
l_mode_optim.append('precision_max_depth')
l_xgb_accuracy.append(xgb_accuracy)
l_best_param.append(best_param)
l_xgb_precision.append(xgb_precision)
l_xgb_recall.append(xgb_recall)

In [ ]:
xgb_precision = 0
xgb_accuracy = 0
xgb_recall = 0
best_param = 0
xgb_optim_value = -9999999
for value_param in range(1,9,1):
    print(value_param)
    param = {'max_depth':value_param, 'eta':0.3, 'silent': 1, 'objective':'binary:logistic', 'eval_metric': 'auc','lambda':1.5} #,'gamma':1.5
    y_output, pred, bst = calcul_model(df_complaint_dataset, 0.3, y_to_predict, param, 300, 50)
    pred_binary =  np.where(pred > 0.5, 1, 0) 
    if _execute_model_utilitie(y_output, pred_binary) >  xgb_optim_value:
        xgb_optim_value = _execute_model_utilitie(y_output, pred_binary)
        optimum_confusion_metrics = metrics.confusion_matrix(y_output.tolist(), pred_binary)
        xgb_accuracy = metrics.accuracy_score(y_output.tolist(), pred_binary)
        best_param = value_param
        xgb_precision = metrics.precision_score(y_output.tolist(), pred_binary)
        xgb_recall = metrics.recall_score(y_output.tolist(), pred_binary)

In [ ]:
l_mode_optim.append('optim_max_depth')
l_xgb_accuracy.append(xgb_accuracy)
l_best_param.append(best_param)
l_xgb_precision.append(xgb_precision)
l_xgb_recall.append(xgb_recall)

In [ ]:
best_param

In [ ]:
l_xgb_precision

In [ ]:
l_xgb_recall

In [ ]:
optimum_confusion_metrics

In [ ]:
import pylab as pl

xgb_precision = 0
xgb_accuracy = 0
xgb_recall = 0
best_param = 0
xgb_optim_value = -9999999
for value_param in np..arange(0.05,0.5,0.05):
    print(value_param)
    param = {'max_depth': 7, 'eta':value_param, 'silent': 1, 'objective':'binary:logistic', 'eval_metric': 'auc','lambda':1.5} #,'gamma':1.5
    y_output, pred, bst = calcul_model(df_complaint_dataset, 0.3, y_to_predict, param, 300, 50)
    pred_binary =  np.where(pred > 0.5, 1, 0) 
    if _execute_model_utilitie(y_output, pred_binary) >  xgb_optim_value:
        xgb_optim_value = _execute_model_utilitie(y_output, pred_binary)
        optimum_confusion_metrics = metrics.confusion_matrix(y_output.tolist(), pred_binary)
        xgb_accuracy = metrics.accuracy_score(y_output.tolist(), pred_binary)
        best_param = value_param
        xgb_precision = metrics.precision_score(y_output.tolist(), pred_binary)
        xgb_recall = metrics.recall_score(y_output.tolist(), pred_binary)

In [ ]:
optimum_confusion_metrics

In [ ]:
xgb_precision

In [ ]:
best_param

In [ ]:
print(metrics.confusion_matrix(y_output.tolist(), pred_binary))

In [ ]:
value = _execute_model_utilitie(y_output, pred_binary)

In [ ]:
print ('accuracy : ' + str(xgb_accuracy))
print ('precision : ' + str(xgb_precision))
print ('recall : ' + str(xgb_recall))

In [ ]:
nb_y0_pred0

In [ ]:
sum(np.where(pred > 0.5, 1, 0))
# y_output.sum()

In [ ]:
print ('')

In [ ]:
np.sum(pred_binary * y_output) +  - 

In [ ]:
20 * np.sum(pred_binary) - np.sum(pred_binary * y_output)

In [ ]:
df_output = pd.DataFrame(np.transpose([np.array(y_output),pred_binary, pred]), columns=['y', 'y_pred_0.5', 'y_pred_log'])

In [ ]:
df_output.to_csv('D:/Mission/Dauphine/2019/20190204/_data/y_pred.csv',sep=';',index=False)

### SVM

In [ ]:
from sklearn import svm
X = [[0, 0], [1, 1]]
y = [0, 1]
clf = svm.SVC(gamma='scale')
clf.fit(X, y) 

### CNN

In [6]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dropout, Flatten, Dense
from keras.optimizers import Adam
from collections import Counter

Using TensorFlow backend.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size=test_size, random_state=rnd)

In [ ]:
print("Nombre de reviews pour l'entraînement : ", np.shape(x_train))
print("Nombre de reviews pour le test : ", np.shape(x_test))
vocab_size = 5000
train = reduce_vocab(x_train,vocab_size)
test = reduce_vocab(x_test,vocab_size)
print("Nombre de reviews pour l'entraînement : ", np.shape(x_train))
print("Nombre de reviews pour le test : ", np.shape(x_test))
vocab_size = 5000
train = reduce_vocab(x_train,vocab_size)
test = reduce_vocab(x_test,vocab_size)
lens = np.array(list(map(len, train)))
print('Maximum text length:', lens.max(),' -- Minimum length:', lens.min(), '-- Mean length of text:',lens.mean())
# Homogénéisation de la taille des commentaires par padding
seq_len = 215
trn = sequence.pad_sequences(train, maxlen = seq_len,value=0)
test = sequence.pad_sequences(test, maxlen = seq_len,value=0)
model = CNN()
model.fit(trn, y_train, validation_data=(test, y_test), epochs=6, batch_size=64)
scores = model.evaluate(test,y_test,verbose=0)
print('loss: ', scores[0],'- accuracy: ', scores[1])

## Local function

In [30]:
def _execute_model_utilitie(y_output, y_pred):
    nb_y1_pred1 = np.sum(pred_binary * y_output)
    nb_y0_pred1 = np.sum(pred_binary) - nb_y1_pred1
    nb_y1_pred0 = sum(y_output) - nb_y1_pred1
    nb_y0_pred0 =  len(y_output) - sum(y_output) - nb_y0_pred1
    return nb_y1_pred1 * 100 + nb_y0_pred0 - 25 * nb_y0_pred1 - 50 * nb_y1_pred0        

In [31]:
def _print_eval_model(y_output, y_pred):
    print('Confusion matrix : \r\n' + str(metrics.confusion_matrix(y_output.tolist(), y_pred)))
    print('Accuracy : ' + str(metrics.accuracy_score(y_output.tolist(), y_pred)))
    print('Precision : ' + str(metrics.precision_score(y_output.tolist(), y_pred)))
    print('Recall : ' + str(metrics.recall_score(y_output.tolist(), y_pred)))

In [19]:
# def _lemmatize(x):
#     l_lemmatized = []
#     for word in tokenizer.tokenize(x):
#         word_postag = nltk.pos_tag(tokenizer.tokenize(word))[0]
#         if 'VB' in word_postag[1]:
#             l_lemmatized.append(lemmatizer.lemmatize(word_postag[0], 'v'))
#         elif 'NN' in word_postag[1]:
#             l_lemmatized.append(lemmatizer.lemmatize(word_postag[0], 'n'))
#         elif 'JJ' in word_postag[1]:
#             l_lemmatized.append(lemmatizer.lemmatize(word_postag[0], 'a'))
#         elif 'RB' in word_postag[1]:
#             try:
#                 if len(wn.synset(word_postag[0]+'.r.1').lemmas()[0].pertainyms()) > 0:                
#                     l_lemmatized.append(wn.synset(word_postag[0]+'.r.1').lemmas()[0].pertainyms()[0].name())
#                 else:
#                     l_lemmatized.append(word_postag[0])
#             except:
#                 l_lemmatized.append(word_postag[0])
#                 continue                    
#         else: 
#             l_lemmatized.append(word_postag[0])
#     return ' '.join(l_lemmatized)

In [29]:
def _lemmatize(x):
    l_lemmatized = []
    for word in tokenizer.tokenize(x):
        wn_word = wn.synsets(word)
        if len(wn_word) > 0:
            l_lemmatized.append(lemmatizer.lemmatize(word, wn_word[0].pos()))
        else: 
            l_lemmatized.append(word)
    return ' '.join(l_lemmatized)

In [32]:
def _nb_word_dispute(x):
    return len(set(tokenizer.tokenize(x)).intersection(set_disputed))

In [33]:
def CNN():
    model = Sequential()
    model.add(Embedding(vocab_size, 32, input_length=seq_len))
    
    model.add(Dropout(0.2))
    model.add(Conv1D(64, 5, padding='same', activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling1D())
    
    model.add(Flatten())
    
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['recall', 'precision'])
    print(model.summary())
    
    return model

In [34]:
def get_text(data_ids, len_data):
    '''
    Correspondance entre les mots et leur codage via un entier
    '''
    word_to_id = imdb.get_word_index()
    word_to_id = {k:(v+3) for k,v in word_to_id.items()}
    word_to_id["<PAD>"] = 0
    word_to_id["<START>"] = 1
    word_to_id["<UNK>"] = 2
    id_to_word = {value:key for key,value in word_to_id.items()}

    text = []
    for i in range(len_data):
        text.append(' '.join(id_to_word[id] for id in data_ids[i]))
    return text

In [35]:
def reduce_vocab(data,vocab_size):
    '''
    Supprime les mots qui ne font pas partie du dictionnaire des vocab_size mots les plus utilisés
    '''
    data = [list(filter(lambda x: x <=vocab_size-1, list_)) for list_ in data]
    return data

In [36]:
def get_label_txt(data):
    '''
    Correspondance entre le type d'avis et son codage binaire
    '''
    values = []
    for idx in range(len(data)):
        if data[idx] == [1]:
            values.append('Positive')
        else:
            values.append('Negative')
    return values